## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import OrderedDict 
from sklearn import metrics, model_selection
from torch.optim import Adam

sys.path.append(os.path.abspath(''))

import utils.more_torch_functions as mtf

from compiling_nn.build_odd import compile_nn
from datasets.loan import get_loan_dataset
from utils.custom_activations import StepActivation, StepFunction
from utils.modules import Parallel, MaxLayer
from utils.custom_loss import AsymBCELoss

torch.autograd.set_detect_anomaly(True)
torch.autograd.gradcheck
pd.options.mode.copy_on_write = True

## Load data

In [2]:
np_x, np_y = get_loan_dataset(balancing=True, discretizing=False, hot_encoding=True, rmv_pct=0.985)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([280, 14])


## Metrics

In [3]:
def cm(y_true, y_pred):
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=[False, True])
    return cm_display

def plot_cm(y_true, y_pred):
    cm_display = cm(y_true, y_pred)
    _, ax = plt.subplots(1, 1, figsize=(4,8))
    cm_display.plot(ax=ax, colorbar=False)

def plot_combine_cm(cms, titles=None):
    n = len(cms)
    fig, axs = plt.subplots(1, n, figsize=(4*n, 8))
    if titles:
        for ax, cm, title in zip(axs, cms, titles):
            cm.plot(ax=ax, colorbar=False)
            ax.set_title(title)
    else:
        for ax, cm in zip(axs, cms):
            cm.plot(ax=ax, colorbar=False)
    fig.tight_layout()

def cov_score(y_true, y_pred):
    labels = np.unique(y_true)
    scores = {}

    for label in labels:
        indices_true = np.where(y_true == label)[0]
        indices_pred = np.where(y_pred == label)[0]
        scores[label] = len(np.intersect1d(indices_true, indices_pred))/len(indices_true)

    return scores

def train_model(x, y, model, loss_fn, optimizer, max_epoch):
    for _ in range(max_epoch):
        model.train()
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)

        model.zero_grad()
        loss.backward()
        optimizer.step()

    return y_pred

def cross_valid(X, Y, model, loss_fn, optimizer, skf, **kw_train):
    for train_index, test_index in skf.split(X, Y):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        mtf.reset_model(model)
        y_pred = train_model(x_train, y_train, model, loss_fn, optimizer, **kw_train)
        y_pred_train = y_pred.detach().round()
        model.eval()
        y_pred_eval = model(x_test).detach()
        yield y_pred_train, y_train, y_pred_eval, y_test

def tnot(a): return torch.logical_not(a)
def tor(a,b): return torch.logical_or(a,b)
def tand(a,b): return torch.logical_and(a,b)
def txor(a,b): return torch.logical_xor(a,b)

## Networks

### Network parts

In [4]:
class ApproxNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        hl1 = 10

        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,1)),
            ('a2', StepActivation())
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x

class CentralNet(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.Sigmoid()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.Sigmoid()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

### Previous Network (and related)

In [5]:
class NetResults():
    def __init__(self, *tensors):
        for tensor in tensors:
            self.register_result(tensor)

    def __getattr__(self, name):
        if hasattr(self.x, name):
            return getattr(self.x, name)
        else:
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")
    
    def __dir__(self):
        return dir(self.x)

    def __str__(self):
        return '\n'.join([str(t) for t in self.tensors()])

    def tensors(self):
        for v in self.__dict__.values():
            yield v

    def detach(self):
        for t in self.tensors():
            t.detach()
        return self
    
    def round(self, *args):
        for t in self.tensors():
            t.round(*args)
        return self

class Netv1(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.a1 = ApproxNet()
        self.a2 = ApproxNet()
        self.nn = CentralNet()

    def forward(self, x):
        xa1 = self.a1(x)
        xa2 = self.a2(x)
        xnn = self.nn(x)

        res = [xnn, xa1, xa2]

        # /!\ to change for backward propagation /!\
        x = mtf.bitwise_big_or(*[(torch.round(t)).to(bool) for t in res])
        # maximum ???
        # xmax = mtf.maximum(res)
        # x = torch.where(xmax > 0.5, xmax, xnn)

        x = NetResults(x, *res)

        return x

### New Network definition

In [6]:
class Netv2(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                ('nn', CentralNet()),
                ('approx1', ApproxNet()),
                ('approx2', ApproxNet()),
            ]))),
            ('or', MaxLayer()),
        ]))

    def forward(self, input):
        return self.net(input)

intermediate_outputs = {}
def get_intermediate_outputs(name):
    def hook(model, input, output):
        if model.training:
            intermediate_outputs.setdefault(name, dict())["train"] = output
        else:
            intermediate_outputs.setdefault(name, dict())["valid"] = output
    return hook

## Network evaluation

In [7]:
model = Netv2()
loss_fn = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)

model.net.nets.register_forward_hook(get_intermediate_outputs("parallel_out"))

skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=104)
bnet_split_res = cross_valid(x_data, y_data, model, loss_fn, optimizer, skf, max_epoch=5000)

dict_metrics = {(modelname, metric, key): [] for modelname in ("net", "nn", "a1net", "a2net")
                for metric in ("f1score", "coverage0", "coverage1") for key in ("valid", "train")}

for i, (train_pred, train_true, valid_pred, valid_true) in enumerate(bnet_split_res):
    out_nns = intermediate_outputs["parallel_out"]
    for d in out_nns.values():
        for k, v in d.items():
            d[k] = v.detach().round()

    prompts = []
    pn = 5
    sep_model = f"{'|':^9}"
    for k, pred, true in [["valid", valid_pred, valid_true], ["train", train_pred, train_true]]:
        net_f1_score = metrics.f1_score(true, pred, average="binary")
        nn_f1_score = metrics.f1_score(true, out_nns[k]["nn"], average="binary")
        prompts.append(f"{'Net':<15}{net_f1_score:.3f}{sep_model}{'CentralNet':<15}{nn_f1_score:.3f}")
        dict_metrics[('net', 'f1score', k)].append(net_f1_score)
        dict_metrics[('nn', 'f1score', k)].append(nn_f1_score)

        a1_f1_score = metrics.f1_score(true, out_nns[k]["approx1"], average="binary")
        a2_f1_score = metrics.f1_score(true, out_nns[k]["approx2"], average="binary")
        prompts.append(f"{'Approx 1':<15}{a1_f1_score:.3f}{sep_model}{'Approx 2':<15}{a2_f1_score:.3f}")
        dict_metrics[('a1net', 'f1score', k)].append(a1_f1_score)
        dict_metrics[('a2net', 'f1score', k)].append(a2_f1_score)

        nn_cov_score = cov_score(true, out_nns[k]["nn"])
        a1_cov_score = cov_score(true, out_nns[k]["approx1"])
        a2_cov_score = cov_score(true, out_nns[k]["approx2"])
        prompts.append(f"{'CentralNet':<15}{nn_cov_score[0]:.3f}{sep_model}{'CentralNet':<15}{nn_cov_score[1]:.3f}")
        prompts.append(f"{'Approx 1':<15}{a1_cov_score[0]:.3f}{sep_model}{'Approx 1':<15}{a1_cov_score[1]:.3f}")
        prompts.append(f"{'Approx 2':<15}{a2_cov_score[0]:.3f}{sep_model}{'Approx 2':<15}{a2_cov_score[1]:.3f}")
        dict_metrics[('nn', 'coverage1', k)].append(nn_cov_score[1])
        dict_metrics[('a1net', 'coverage1', k)].append(a1_cov_score[1])
        dict_metrics[('a2net', 'coverage1', k)].append(a2_cov_score[1])
        dict_metrics[('nn', 'coverage0', k)].append(nn_cov_score[0])
        dict_metrics[('a1net', 'coverage0', k)].append(a1_cov_score[0])
        dict_metrics[('a2net', 'coverage0', k)].append(a2_cov_score[0])

    sep_tv = f"{'||':^10}"
    print(f"Fold {i+1:3} :            {'Valid':^49}{sep_tv}{'Train':^49}",
          f"\tF1 score      {prompts[0]}{sep_tv}{prompts[pn+0]}",
          f"\t              {prompts[1]}{sep_tv}{prompts[pn+1]}",
          f"\tCoverage      {prompts[2]}{sep_tv}{prompts[pn+2]}",
          f"\t              {prompts[3]}{sep_tv}{prompts[pn+3]}",
          f"\t              {prompts[4]}{sep_tv}{prompts[pn+4]}",
          sep='\n')

Fold   1 :                                  Valid                          ||                          Train                      
	F1 score      Net            0.688    |    CentralNet     0.690    ||    Net            1.000    |    CentralNet     0.996
	              Approx 1       0.381    |    Approx 2       0.421    ||    Approx 1       0.880    |    Approx 2       0.839
	Coverage      CentralNet     0.643    |    CentralNet     0.714    ||    CentralNet     1.000    |    CentralNet     0.992
	              Approx 1       0.786    |    Approx 1       0.286    ||    Approx 1       1.000    |    Approx 1       0.786
	              Approx 2       0.929    |    Approx 2       0.286    ||    Approx 2       1.000    |    Approx 2       0.722
Fold   2 :                                  Valid                          ||                          Train                      
	F1 score      Net            0.600    |    CentralNet     0.643    ||    Net            1.000    |    CentralNet     

In [8]:
df_metrics = pd.DataFrame.from_dict(dict_metrics, orient='index')
mean_metrics = df_metrics.mean(axis=1)
mean_prompts = [
    f"{'Net':<15}{mean_metrics[('net', 'f1score', 'valid')]:.3f}{sep_model}{'CentralNet':<15}{mean_metrics[('nn', 'f1score', 'valid')]:.3f}",
    f"{'Approx 1':<15}{mean_metrics[('a1net', 'f1score', 'valid')]:.3f}{sep_model}{'Approx 2':<15}{mean_metrics[('a2net', 'f1score', 'valid')]:.3f}",
    f"{'CentralNet':<15}{mean_metrics[('nn', 'coverage0', 'valid')]:.3f}{sep_model}{'CentralNet':<15}{mean_metrics[('nn', 'coverage1', 'valid')]:.3f}",
    f"{'Approx 1':<15}{mean_metrics[('a1net', 'coverage0', 'valid')]:.3f}{sep_model}{'Approx 1':<15}{mean_metrics[('a1net', 'coverage1', 'valid')]:.3f}",
    f"{'Approx 2':<15}{mean_metrics[('a2net', 'coverage0', 'valid')]:.3f}{sep_model}{'Approx 2':<15}{mean_metrics[('a2net', 'coverage1', 'valid')]:.3f}",
    f"{'Net':<15}{mean_metrics[('net', 'f1score', 'train')]:.3f}{sep_model}{'CentralNet':<15}{mean_metrics[('nn', 'f1score', 'train')]:.3f}",
    f"{'Approx 1':<15}{mean_metrics[('a1net', 'f1score', 'train')]:.3f}{sep_model}{'Approx 2':<15}{mean_metrics[('a2net', 'f1score', 'train')]:.3f}",
    f"{'CentralNet':<15}{mean_metrics[('nn', 'coverage0', 'train')]:.3f}{sep_model}{'CentralNet':<15}{mean_metrics[('nn', 'coverage1', 'train')]:.3f}",
    f"{'Approx 1':<15}{mean_metrics[('a1net', 'coverage0', 'train')]:.3f}{sep_model}{'Approx 1':<15}{mean_metrics[('a1net', 'coverage1', 'train')]:.3f}",
    f"{'Approx 2':<15}{mean_metrics[('a2net', 'coverage0', 'train')]:.3f}{sep_model}{'Approx 2':<15}{mean_metrics[('a2net', 'coverage1', 'train')]:.3f}",
]
print(f"Average  :            {'Valid':^49}{sep_tv}{'Train':^49}",
          f"\tF1 score      {mean_prompts[0]}{sep_tv}{mean_prompts[pn+0]}",
          f"\t              {mean_prompts[1]}{sep_tv}{mean_prompts[pn+1]}",
          f"\tCoverage      {mean_prompts[2]}{sep_tv}{mean_prompts[pn+2]}",
          f"\t              {mean_prompts[3]}{sep_tv}{mean_prompts[pn+3]}",
          f"\t              {mean_prompts[4]}{sep_tv}{mean_prompts[pn+4]}",
          sep='\n')

Average  :                                  Valid                          ||                          Train                      
	F1 score      Net            0.757    |    CentralNet     0.697    ||    Net            1.000    |    CentralNet     0.957
	              Approx 1       0.622    |    Approx 2       0.633    ||    Approx 1       0.893    |    Approx 2       0.888
	Coverage      CentralNet     0.736    |    CentralNet     0.679    ||    CentralNet     1.000    |    CentralNet     0.921
	              Approx 1       0.779    |    Approx 1       0.564    ||    Approx 1       0.999    |    Approx 1       0.813
	              Approx 2       0.729    |    Approx 2       0.600    ||    Approx 2       1.000    |    Approx 2       0.804
